In [3]:
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell
import os
import numpy as np  # Mejora: Importing numpy


def create_jupyter_notebook():
    """
    Creates a Jupyter Notebook with specified content and structure.

    Args:
        None

    Returns:
        None

    Assumptions:
        - The necessary libraries are installed.
        - The notebook will be created in the current directory.
    """
    nb = new_notebook()

    # --- Libraries ---
    nb.cells.append(new_markdown_cell("# Librerías"))
    nb.cells.append(
        new_code_cell(
            """
import numpy as np
import timber_nds.essentials as essentials
from timber_nds.calculation import WeightCalculator, effective_length, radius_of_gyration, polar_moment_of_inertia, RectangularSectionProperties
from timber_nds.design import SummarizeFactors
    """
        )
    )

    # --- Parameters ---
    nb.cells.append(new_markdown_cell("# Parámetros"))
    nb.cells.append(
        new_markdown_cell(
            """
A continuación, se definen todos los parámetros necesarios para el cuaderno, incluyendo factores de reducción de resistencia, propiedades de materiales y dimensiones geométricas.

**Factores de Reducción de Resistencia:**

*   `phi_tension`: Factor de reducción de resistencia para tensión.
*   `phi_bending`: Factor de reducción de resistencia para flexión.
*   `phi_shear`: Factor de reducción de resistencia para corte.
*   `phi_compression`: Factor de reducción de resistencia para compresión.
*    `phi_perp_compression`: Factor de reducción de resistencia para compresión perpendicular a la fibra.

**Propiedades del Material:**

*   `species`: Especie de madera.
*   `specific_gravity`: Gravedad específica de la madera.
*   `fibre_saturation_point`: Punto de saturación de la fibra de la madera (%).
*   `moisture_content`: Contenido de humedad de la madera (%).
*   `e_modulus`: Módulo de elasticidad de la madera (Pa).
*  `fb`: Resistencia a la flexión (Pa).
* `ft`: Resistencia a la tracción (Pa).
* `fv`: Resistencia al corte (Pa).
* `fc`: Resistencia a la compresión paralela a la fibra (Pa).
* `fc_perp`: Resistencia a la compresión perpendicular a la fibra (Pa).

**Dimensiones Geométricas:**

*   `width`: Ancho de la sección transversal (m).
*   `depth`: Profundidad de la sección transversal (m).
*   `length`: Longitud del elemento (m).
*   `k_factor`: Factor de longitud efectiva.

**Factores de Ajuste:**

*   Se definen varios factores de ajuste para tensión, flexión, corte, compresión, compresión perpendicular a la fibra y módulo de elasticidad.
"""
        )
    )
    nb.cells.append(
        new_code_cell(
            """
# Factors
phi_tension = 0.90
phi_bending = 0.85
phi_shear = 0.75
phi_compression = 0.90
phi_perp_compression = 0.65

# Material Properties
species = "Douglas Fir-Larch"
specific_gravity = 0.50
fibre_saturation_point = 28
moisture_content = 19
e_modulus = 12400000000 # Pa
fb = 25000000 # Pa
ft = 15000000 # Pa
fv = 3000000 # Pa
fc = 17000000 # Pa
fc_perp = 4000000 # Pa


# Geometry
width = 0.150 # m
depth = 0.300 # m
length = 4 # m
k_factor = 1.0

#Adjustment Factors
#Tension
ft_factors = essentials.TensionAdjustmentFactors(
    load_duration_factor = 1.0,
    wet_service_factor = 0.85,
    temperature_factor = 1.0,
    size_factor = 1.0,
    incising_factor = 1.0
)
#Bending
fb_factors = essentials.BendingAdjustmentFactors(
    load_duration_factor = 1.0,
    wet_service_factor = 0.85,
    temperature_factor = 1.0,
    size_factor = 1.0,
    lateral_stability_factor = 1.0,
    beam_stability_factor = 1.0,
    incising_factor = 1.0
)
#Shear
fv_factors = essentials.ShearAdjustmentFactors(
    load_duration_factor = 1.0,
    wet_service_factor = 0.97,
    temperature_factor = 1.0,
    incising_factor = 1.0
)
#Compression
fc_factors = essentials.CompressionAdjustmentFactors(
    load_duration_factor = 1.0,
    wet_service_factor = 0.80,
    temperature_factor = 1.0,
    column_stability_factor = 1.0,
    incising_factor = 1.0
)
#Compression Perpendicular
fc_perp_factors = essentials.PerpendicularAdjustmentFactors(
    wet_service_factor = 0.67,
    temperature_factor = 1.0,
    bearing_area_factor = 1.0,
    incising_factor = 1.0
)
#Elastic Modulus
e_factors = essentials.ElasticModulusAdjustmentFactors(
    wet_service_factor = 0.90,
    temperature_factor = 1.0,
    incising_factor = 1.0
)
    """
        )
    )
    # --- Example ---
    nb.cells.append(new_markdown_cell("# Ejemplo"))
    nb.cells.append(
        new_markdown_cell(
            """
Este cuaderno demostrará el cálculo del peso, las propiedades geométricas y la capacidad de una viga de madera de sección rectangular. 
Comenzaremos calculando el peso de la viga, luego sus propiedades geométricas, y finalmente, integraremos todos los resultados para calcular la capacidad de la viga.
El ejemplo se desarrollará de manera continua a través de todas las secciones, ilustrando el flujo de trabajo típico en el diseño de estructuras de madera.
"""
        )
    )

    # --- Function Examples ---
    # Function 1: Weight Calculation
    nb.cells.append(new_markdown_cell("## Ejemplo de Función 1: Cálculo del Peso"))
    nb.cells.append(
        new_markdown_cell(
            """
Esta sección demuestra cómo calcular el peso de un elemento de madera utilizando la clase `WeightCalculator`.
Primero, se crea una instancia de `WoodMaterial`, `RectangularSection` y `MemberDefinition` con los parámetros definidos anteriormente.
Luego, se crea una instancia de `WeightCalculator` y se calcula el peso del elemento para el contenido de humedad especificado.
"""
        )
    )
    nb.cells.append(
        new_code_cell(
            """
# Creating instances of classes
wood_material = essentials.WoodMaterial(
    species = species,
    specific_gravity = specific_gravity,
    fibre_saturation_point = fibre_saturation_point,
    e_modulus = e_modulus,
    fb = fb,
    ft = ft,
    fv = fv,
    fc = fc,
    fc_perp = fc_perp
    )
section = essentials.RectangularSection(width,depth)
element = essentials.MemberDefinition(length)
weight_calculator = WeightCalculator(wood_material, section, element)
element_weight = weight_calculator.calculate_weight_at_moisture_content(moisture_content)

print(f"El peso del elemento de madera es: {element_weight:.2f} kg")
    """
        )
    )

    # Function 2: Geometric Properties
    nb.cells.append(new_markdown_cell("## Ejemplo de Función 2: Propiedades Geométricas"))
    nb.cells.append(
        new_markdown_cell(
            """
En esta sección, calculamos las propiedades geométricas de la sección rectangular utilizando la clase `RectangularSectionProperties`.
Se crea una instancia de `RectangularSectionProperties` con el ancho y la profundidad de la sección.
Luego, calculamos el área, los momentos de inercia, los módulos de sección y los radios de giro.
"""
        )
    )
    nb.cells.append(
        new_code_cell(
            """
# Calculate Section Properties
section_properties = RectangularSectionProperties(width, depth)
area = section_properties.area()
i_yy = section_properties.i_yy()
i_zz= section_properties.i_zz()
elastic_section_modulus_yy = section_properties.elastic_section_modulus_yy()
elastic_section_modulus_zz = section_properties.elastic_section_modulus_zz()
radius_gyration_yy = section_properties.radius_of_gyration_yy()
radius_gyration_zz = section_properties.radius_of_gyration_zz()

print(f"Área de la sección: {area:.4f} m^2")
print(f"Momento de inercia Iyy: {i_yy:.6f} m^4")
print(f"Momento de inercia Izz: {i_zz:.6f} m^4")
print(f"Módulo de sección elástico Sxx: {elastic_section_modulus_yy:.6f} m^3")
print(f"Módulo de sección elástico Syy: {elastic_section_modulus_zz:.6f} m^3")
print(f"Radio de giro r_yy: {radius_gyration_yy:.4f} m")
print(f"Radio de giro r_zz: {radius_gyration_zz:.4f} m")
    """
        )
    )

    # Function 3: Effective Length
    nb.cells.append(new_markdown_cell("## Ejemplo de Función 3: Longitud Efectiva"))
    nb.cells.append(
        new_markdown_cell(
            """
Aquí, calculamos la longitud efectiva de la viga usando la función `effective_length`.
La longitud efectiva se calcula multiplicando la longitud real de la viga por el factor de longitud efectiva (K).
"""
        )
    )
    nb.cells.append(
        new_code_cell(
            """
# Calculate Effective Length
effective_length_value = effective_length(k_factor, length)
print(f"Longitud efectiva de la viga: {effective_length_value:.2f} m")
    """
        )
    )

    # Function 4: Combined Factors
    nb.cells.append(new_markdown_cell("## Ejemplo de Función 4: Factores Combinados"))
    nb.cells.append(
        new_markdown_cell(
            """
En esta sección, demostramos cómo utilizar la clase `SummarizeFactors` para calcular los factores combinados.
Se crea una instancia de `SummarizeFactors` con todos los factores de ajuste definidos anteriormente.
Luego, se calcula el factor combinado para cada propiedad mecánica (tensión, flexión, corte, compresión, compresión perpendicular a la fibra y módulo de elasticidad).
"""
        )
    )
    nb.cells.append(
        new_code_cell(
            """
# Calculate Combined Factors
summarize_factors = SummarizeFactors(
    tension_factors = ft_factors,
    bending_factors = fb_factors,
    shear_factors = fv_factors,
    compression_factors = fc_factors,
    compression_perp_factors = fc_perp_factors,
    elastic_modulus_factors = e_factors,
)

combined_factors = summarize_factors.calculate_combined_factors()
print(f"Factores combinados: {combined_factors}")
    """
        )
    )

    # --- Tests ---
    nb.cells.append(new_markdown_cell("# Pruebas"))
    # Test for WeightCalculator.calculate_density_at_moisture_content
    nb.cells.append(
        new_markdown_cell(
            """
### Prueba para `WeightCalculator.calculate_density_at_moisture_content`

```python
import pytest
from timber_nds.calculation import WeightCalculator
import timber_nds.essentials as essentials

def test_calculate_density_at_moisture_content():
    # Arrange
    wood_material = essentials.WoodMaterial(
        species = "Douglas Fir-Larch",
        specific_gravity = 0.50,
        fibre_saturation_point = 28,
        e_modulus = 12400000000,
        fb = 25000000,
        ft = 15000000,
        fv = 3000000,
        fc = 17000000,
        fc_perp = 4000000
    )
    section = essentials.RectangularSection(0.150, 0.300)
    element = essentials.MemberDefinition(4)
    weight_calculator = WeightCalculator(wood_material, section, element)
    moisture_content = 19

    # Act
    density = weight_calculator.calculate_density_at_moisture_content(moisture_content)

    # Assert
    assert density == pytest.approx(576.13, 0.01)

def test_calculate_density_at_moisture_content_invalid_input():
        # Arrange
    wood_material = essentials.WoodMaterial(
        species = "Douglas Fir-Larch",
        specific_gravity = 0.50,
        fibre_saturation_point = 28,
        e_modulus = 12400000000,
        fb = 25000000,
        ft = 15000000,
        fv = 3000000,
        fc = 17000000,
        fc_perp = 4000000
    )
    section = essentials.RectangularSection(0.150, 0.300)
    element = essentials.MemberDefinition(4)
    weight_calculator = WeightCalculator(wood_material, section, element)


    with pytest.raises(TypeError):
        weight_calculator.calculate_density_at_moisture_content("invalid")
    with pytest.raises(ValueError):
        weight_calculator.calculate_density_at_moisture_content(-5)
    with pytest.raises(ValueError):
       wood_material_negative_fsp = essentials.WoodMaterial(
            species = "Douglas Fir-Larch",
            specific_gravity = 0.50,
            fibre_saturation_point = -28,
             e_modulus = 12400000000,
            fb = 25000000,
            ft = 15000000,
            fv = 3000000,
            fc = 17000000,
            fc_perp = 4000000
        )
       weight_calculator_negative_fsp = WeightCalculator(wood_material_negative_fsp, section, element)
       weight_calculator_negative_fsp.calculate_density_at_moisture_content(10)
"""
        )
    )

    # --- Save the Notebook ---
    file_path = "timber_design_notebook.ipynb"  # Claridad: More descriptive file name
    with open(file_path, "w", encoding="utf-8") as f: # Claridad: added encoding for better file compatibility
        nbformat.write(nb, f)
    print(f"Jupyter Notebook created successfully at: {file_path}")

if __name__ == "__main__": # Claridad: standard way of calling main function.
    create_jupyter_notebook()

Jupyter Notebook created successfully at: timber_design_notebook.ipynb
